In [ ]:
from PosSelect_Functions_Old import *
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde
import pandas as pd
import numpy as np
import copy
import seaborn as sns
from scipy.stats import mannwhitneyu as mwu
from scipy.stats import ttest_ind
from scipy.stats import ttest_rel
from statsmodels.stats.multitest import fdrcorrection
from scipy.stats import wilcoxon
from scipy.optimize import curve_fit
from scipy.stats import fisher_exact

hfont = {'fontname':'Arial'}
plt.rcParams["font.family"] = "Arial"

#Code borrowed heavily from here: https://stackoverflow.com/questions/62375034/find-non-overlapping-area-between-two-kde-plots
plt.rcParams.update(
    {"text.usetex": False}
)

d_abrev = {"LiangSteinNeuron":"FC exc. neur.", "FetalChondrocytes":"F chond.", "SertoliMale":"FG sertoli", "preGC_IIaFemale":"FG preGC IIa",\
          "NeuralFemale":"FG neur.", "FetalGonadImmuneFemale":"FG immune", "VIP":"AC VIP inh. neur.", "LiangSteinProgenitor":"FC prog.",\
          "AdultHeartVentricularCardiomyocyte":"AH cardiomyo.", "AdultLoopOfHenle":"AK loop of henle", "FetalBrainNeurGlioblast_CB_VZ":"FCB glioblast",\
         "AdultProximalTubule":"AK prox. tub.", "FetalLeydigMale":"FG leydig", "SST":"AC SST inh neur.", "KosoyRoussosControlMicroglia":"AC microglia",\
         "FetalBrainFloorPlate":"FB fl. plate", "FetalArterialECs":"FH endoth.", "ASCT":"AC astro.", "FetalBrainCOP":"FB COP",\
         "AMY":"AA neur.", "PVALB":"AC PVALB inh neur.", "ITL23":"AC L2-3 IT neur.", "FetalBrainNeurCB_GNP_IPC_1":"FB inter. prog.", "FetalBrainNeurDAergic":"FB DA neur.",\
          "OGC":"AC Oligo.", "D1Pu":"AP D1 inh neur.", "FetalBrainNeurSerotonergic":"FB 5-HT neur.", "FetalBrainNeurDRG_2":"FS DRG neur.",\
          "FetalHeartPericytes":"FH peri.", "FetalHeartEndocardium":"FH endocard.", "FetalHeartCardiacFibroblasts":"FH fibro.", "FetalBrainNeurPurkinje_6":"FCB Purk. inh neur.",\
          "AdultHeartSmoothMuscle":"AH smooth musc.", "FetalBrainRoofPlate":"FB ro. plate"}

In [ ]:
df = pd.read_csv("Effects_WithV_PerGene_FilterNewTEs_SigThresh0.025_Prop0.6_Strong0.75_Thresh2.txt", sep = "\t")
lfsr = pd.read_csv("Lfsr_WithV_PerGene_FilterNewTEs_SigThresh0.025_Prop0.6_Strong0.75_Thresh2.txt", sep = "\t")

signed_lfsr = np.sign(df.sort_index())*-np.log10(lfsr.sort_index())
signed_lfsr["Mean"] = np.mean(signed_lfsr, axis = 1)

signed_lfsr["Max"] = np.max(signed_lfsr, axis = 1)
signed_lfsr["Min"] = np.min(signed_lfsr, axis = 1)

sig_pos_select = signed_lfsr[signed_lfsr["Max"] > -np.log10(0.05)].sort_values("Max")
sig_pos_select["Variance"] = np.var(sig_pos_select.drop(["Mean", "Max", "Min"], axis = 1), axis = 1)/np.mean(sig_pos_select.drop(["Mean", "Max", "Min"], axis = 1), axis = 1)


In [ ]:
#Comparing across different cutoffs
df = pd.read_csv("Effects_WithV_TFBS_FilterNewTEs_SigThresh0.025_Prop0.6_Strong0.25_Thresh2_Cutoff500.txt", sep = "\t")
df2 = pd.read_csv("Effects_WithV_TFBS_FilterNewTEs_SigThresh0.025_Prop0.8_Strong0.25_Thresh2_Cutoff500.txt", sep = "\t")
df2.columns = [x + " 0.8" for x in df2.columns]
dff = df.join(df2)

out = []

for ct in d_abrev.keys():
    
    out.append([d_abrev[ct], pearsonr(dff[ct], dff[ct + " 0.8"])[0]])
    print(pearsonr(dff[ct], dff[ct + " 0.8"]))
    if ct == "LiangSteinNeuron":
        fig, ax = plt.subplots(figsize = (6, 6))
        sns.regplot(x=dff[ct], y=dff[ct + " 0.8"])
        plt.title("Fetal cortical excitatory neurons", size = 18)
        plt.xlabel("MASH effect size, 60th percentile", size = 16)
        plt.ylabel("MASH effect size, 80th percentile", size = 16)
        plt.xticks(size = 12)
        plt.yticks(size = 12)

In [ ]:
dfp = pd.DataFrame(out)

fig, ax = plt.subplots(figsize = (18, 6))

dfp.columns = ["Cell type", "Pearson's rho"]
sns.barplot(data = dfp, x = "Cell type", y = "Pearson's rho")

plt.ylabel("Pearson's rho", size = 22)
plt.xlabel("Cell type", size = 22)
plt.xticks(size = 16)
plt.yticks(size = 16)

plt.ylim([0, 1])
plt.title("Correlation between MASH effect size for 60th and 80th percentiles", size = 26)
plt.xticks(rotation = 90)

In [ ]:
df = pd.read_csv("Effects_WithV_PerGene_FilterNewTEs_SigThresh0.025_Prop0.6_Strong0.75_Thresh2.txt", sep = "\t")
lfsr = pd.read_csv("Lfsr_WithV_PerGene_FilterNewTEs_SigThresh0.025_Prop0.6_Strong0.75_Thresh2.txt", sep = "\t")

signed_lfsr = np.sign(df.sort_index())*-np.log10(lfsr.sort_index())
signed_lfsr["Mean"] = np.mean(signed_lfsr, axis = 1)

signed_lfsr["Max"] = np.max(signed_lfsr, axis = 1)
signed_lfsr["Min"] = np.min(signed_lfsr, axis = 1)

sig_pos_select = signed_lfsr[signed_lfsr["Max"] > -np.log10(0.05)].sort_values("Max")
sig_pos_select["Variance"] = np.var(sig_pos_select.drop(["Mean", "Max", "Min"], axis = 1), axis = 1)/np.mean(sig_pos_select.drop(["Mean", "Max", "Min"], axis = 1), axis = 1)

#Plot things on volcano plot, modified this to do TFBS
out = []
for index, row in df.iterrows():
    out.append([index, df.columns[np.argmax(list(row))], max(list(row)), np.argmax(list(row))])
dff = pd.DataFrame(out)
dff.index = dff[0]
dff.columns = ["Gene", "Cell type of max", "Effect size", "Argmax"]
out = []
for index, row in lfsr.iterrows():
    out.append([index, list(row)[dff.loc[index][3]]])

dfl = pd.DataFrame(out)
dfl = dfl.set_index(0)
dfl.columns = ["LFSR"]
dff = dff.join(dfl)
new_lfsr = []
for index, row in dff.iterrows():
    if row["Effect size"] < 0:
        new_lfsr.append(1)
    else:
        new_lfsr.append(row["LFSR"])
dff["LFSR"] = new_lfsr
dff["-Log$_{10}$(LFSR)"] = -np.log10(dff["LFSR"])

dff = dff[dff["Effect size"] > 0]
k = []
for index, row in dff.iterrows():
    if row["LFSR"] < 0.05:
        k.append("LFSR < 0.05")
    else:
        k.append("Not significant")
dff['Significance'] = k
fig, ax = plt.subplots(figsize = (6, 4))
sns.scatterplot(data = dff, x = "Effect size", y = "-Log$_{10}$(LFSR)", hue = "Significance", palette = {"LFSR < 0.05":"red", "Not significant":"grey"})
plt.title("Positive selection on CA for individual genes", size = 16)
plt.ylabel("-Log$_{10}$(LFSR)", size = 14)
plt.xlabel("MASH effect size", size = 14)
plt.xticks(size = 11)
plt.yticks(size = 11)
plt.legend(fontsize = 12)

In [ ]:
#Testing for enrichment of neanderthal nonsyn across cutoffs
for fdr_cut in [0.01, 0.05, 0.1]:
    for daf in [0.99, 0.995, 0.999]:
        

        sig_all = signed_lfsr[(signed_lfsr["Max"] > -np.log10(fdr_cut))].sort_values("Max")

        mis = pd.read_csv("Missense_Neanderthal.csv")
        mis = mis[mis["human_DAF"] > daf]
        mis_keep = list(mis["Gene name"])
        genes = []
        for gene in mis_keep:
            if ";" in gene:
                genes = genes + gene.split(";")
            else:
                genes.append(gene)
        mis_sig = len(np.intersect1d(mis_keep, sig_all.index))
        mis_nsig = len(np.intersect1d(np.setdiff1d(mis_keep, sig_all.index), signed_lfsr.index))
        nmis_sig = len(np.setdiff1d(sig_all.index, mis_keep))
        nmis_nsig = len(np.setdiff1d(signed_lfsr.index, mis_keep + list(sig_all.index)))
        print(fdr_cut, daf, fisher_exact([[mis_sig, mis_nsig], [nmis_sig, nmis_nsig]]), [[mis_sig, mis_nsig], [nmis_sig, nmis_nsig]])
        print(np.intersect1d(mis_keep, sig_all.index))

In [ ]:
#Plotting the odds ratios and confidence intervals, used fisher.test in R

fig, ax = plt.subplots(figsize = (8, 4))

# Two sided
daf1 = [4.358369, 1.356663, 10.842554]
daf2 = [5.569823, 1.724064, 13.978426]
daf3 = [7.536774, 1.935983, 21.190395]
categories = ['DAF > 0.99', 'DAF > 0.995', 'DAF > 0.999']
values = [daf1[0], daf2[0], daf3[0]]
lower_ci = [daf1[0] - daf1[1], daf2[0] - daf2[1], daf3[0] - daf3[1]]  # Lower confidence intervals
upper_ci = [daf1[2] - daf1[0], daf2[2] - daf2[0], daf3[2] - daf3[0]]
# Create the bar plot
plt.barh(categories, values, color=["#F42FF51A", "#F42FF51A", "#F42FF51A"], label='Values', alpha = 0.5)

# Add lower confidence intervals as error bars
for i, (x, y, ci) in enumerate(zip(categories, values, lower_ci)):
    plt.hlines(x, y - ci, y, color='black', label='Lower CI' if i == 0 else "")
    plt.vlines(y - ci, i-0.125, i+0.125, color='black')
    plt.hlines(x, y, y + ci, color='black', label='Lower CI' if i == 0 else "")
    plt.vlines(y + ci, i-0.125, i+0.125, color='black')
    #if i == 0:
    #    plt.text(i - 0.025, y-0.025, "*", size = 20)

# Customize the plot
plt.ylabel('Derived allele freq. cutoff', size = 16)
plt.xlabel('Odds ratio', size = 16)
plt.title('Enrichment for human-specific nonsynonymous substitutions', size = 16)
plt.legend([], [], frameon = False)
plt.yticks(size = 14, rotation = 0)
plt.xticks(size = 14, rotation = 0)

plt.show()



In [ ]:
fdr_cut = 0.05
daf = 0.999

sig_all = signed_lfsr[(signed_lfsr["Max"] > -np.log10(fdr_cut))].sort_values("Max")

mis = pd.read_csv("Missense_Neanderthal.csv")
mis = mis[mis["human_DAF"] > daf]
mis_keep = list(mis["Gene name"])
mis_sig = len(np.intersect1d(mis_keep, sig_all.index))
mis_nsig = len(np.intersect1d(np.setdiff1d(mis_keep, sig_all.index), signed_lfsr.index))
nmis_sig = len(np.setdiff1d(sig_all.index, mis_keep))
nmis_nsig = len(np.setdiff1d(signed_lfsr.index, mis_keep + list(sig_all.index)))
print(fdr_cut, daf, fisher_exact([[mis_sig, mis_nsig], [nmis_sig, nmis_nsig]]), [[mis_sig, mis_nsig], [nmis_sig, nmis_nsig]])

np.intersect1d(mis_keep, sig_all.index)

In [ ]:
#Exploring positive selection on TFBS
df = pd.read_csv("Effects_WithV_TFBS_FilterNewTEs_SigThresh0.025_Prop0.6_Strong0.25_Thresh2_Cutoff500.txt", sep = "\t")
lfsr = pd.read_csv("Lfsr_WithV_TFBS_FilterNewTEs_SigThresh0.025_Prop0.6_Strong0.25_Thresh2_Cutoff500.txt", sep = "\t")

signed_lfsr = np.sign(df.sort_index())*-np.log10(lfsr.sort_index())
signed_lfsr["Mean"] = np.mean(signed_lfsr, axis = 1)

signed_lfsr["Max"] = np.max(signed_lfsr, axis = 1)
signed_lfsr["Min"] = np.min(signed_lfsr, axis = 1)

sig_pos_select = signed_lfsr[signed_lfsr["Max"] > -np.log10(0.05)].sort_values("Max")
sig_pos_select["Variance"] = np.var(sig_pos_select.drop(["Mean", "Max", "Min"], axis = 1), axis = 1)/np.mean(sig_pos_select.drop(["Mean", "Max", "Min"], axis = 1), axis = 1)
sig_pos_select

In [ ]:
#Getting ready to plot for TFBS
out = []
for index, row in df.iterrows():
    out.append([index, df.columns[np.argmax(list(row))], max(list(row)), np.argmax(list(row))])
dff = pd.DataFrame(out)
dff.index = dff[0]
dff.columns = ["Gene", "Cell type of max", "Effect size", "Argmax"]
out = []
for index, row in lfsr.iterrows():
    out.append([index, list(row)[dff.loc[index][3]]])

dfl = pd.DataFrame(out)
dfl = dfl.set_index(0)
dfl.columns = ["LFSR"]
dff = dff.join(dfl)
new_lfsr = []
for index, row in dff.iterrows():
    if row["Effect size"] < 0:
        new_lfsr.append(1)
    else:
        new_lfsr.append(row["LFSR"])
dff["LFSR"] = new_lfsr
dff["-Log$_{10}$(LFSR)"] = -np.log10(dff["LFSR"])
dff.sort_values("Effect size").tail(50)

In [ ]:
#Plotting for TFBS
dff = dff[dff["Effect size"] > 0]
k = []
for index, row in dff.iterrows():
    if row["LFSR"] < 0.05:
        k.append("LFSR < 0.05")
    else:
        k.append("Not significant")
dff['Significance'] = k
fig, ax = plt.subplots(figsize = (6, 4))
sns.scatterplot(data = dff, x = "Effect size", y = "-Log$_{10}$(LFSR)", hue = "Significance", palette = {"LFSR < 0.05":"red", "Not significant":"grey"})
plt.title("Positive selection on CA for TFBS", size = 16)
plt.ylabel("-Log$_{10}$(LFSR)", size = 14)
plt.xlabel("MASH effect size", size = 14)
plt.xticks(size = 11)
plt.yticks(size = 11)
plt.legend(fontsize = 12)

In [ ]:
#Plotting individual TFBS
to_plot = "MA1587.1_ZNF135"

sig = pd.DataFrame(sig_pos_select.loc[to_plot])

colors = []
for index, row in sig.iterrows():
    if row[0] > -np.log10(0.05):
        colors.append("#F42FF5")
    else:
        colors.append("#40A94D")
    if len(colors) == 34:
        break

psd = pd.read_csv("Psd_WithV_TFBS_FilterNewTEs_SigThresh0.025_Prop0.6_Strong0.25_Thresh2_Cutoff500.txt", sep = "\t")
eff = pd.read_csv("Effects_WithV_TFBS_FilterNewTEs_SigThresh0.025_Prop0.6_Strong0.25_Thresh2_Cutoff500.txt", sep = "\t")

categories = [d_abrev[x.replace("_Alu_HPO.txt", "")] for x in list(eff.columns)]
values = list(eff.loc[to_plot])
std_dev = [x for x in list(psd.loc[to_plot])]
# Create the bar plot
fig, ax = plt.subplots(figsize = (10, 4))
bars = ax.bar(categories, values, yerr=std_dev, capsize=3, color=colors, alpha=0.9)

# Add labels and title
ax.set_ylabel('MASHR effect size', size = 16)
ax.set_title('ZNF135 binding sites', size = 20)
ax.set_xticks(np.arange(len(categories)))
ax.set_xticklabels(categories, rotation = 90, size = 12)
ax.set_xlabel("Cell type", size = 16)
ax.set_xlim([-0.7, 33.7])
# Show the plot
plt.show()